<a href="https://colab.research.google.com/github/coordanalisis-sedatu/datavivi/blob/main/siete_elementos_viv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##Borrar datos del entorno
rm(list=ls())

In [ ]:
##Crear folders de almacenamiento
dir.create("microdatos", showWarnings = F)

In [ ]:
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse,srvyr,googledrive,janitor,survey,
               sf,httr,jsonlite)

Loading required package: pacman



In [ ]:
options(timeout = 1000)


In [ ]:
#Función para descargar y unzip
download_and_unzip <- function(url, dir) {
  # Crear directorio si no existe
  if (!dir.exists(dir)) {
    dir.create(dir, showWarnings = FALSE)
  }

  # Crear el archivo destino
  destfile <- paste0(dir, "/", basename(url))

  # Descarga
  download.file(url, destfile, mode = "wb")

  # Unzip
  unzip(destfile, exdir = dir)
}


In [ ]:
download_and_unzip("https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_viviendas_csv.zip", "microdatos")
download_and_unzip("https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_hogares_csv.zip", "microdatos")


In [ ]:
# Carga y limpia el conjunto de datos de viviendas para el año 2022
enigh22 <- read.csv("microdatos/viviendas.csv") %>%
  janitor::clean_names() %>%  #Poner las variables en minusculas y quitar las variables
  mutate(
    mat_pared = as.numeric(mat_pared),
    mat_pisos = as.numeric(mat_pisos),
    mat_techos = as.numeric(mat_techos),
    rezago = case_when(
      (tot_resid / num_cuarto) > 2.5 |   # Hacinamiento
        mat_pared %in% c(1, 2, 3, 4, 5, 6) |
        mat_pisos %in% c(1) |
        mat_techos %in% c(1, 2, 3, 4, 6, 7, 9) |
        excusado == 2 ~ "En rezago",
      TRUE ~ "Fuera de rezago"
    )
  )

Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `mat_pisos = as.numeric(mat_pisos)`.
Caused by warning:
! NAs introduced by coercion"


In [ ]:
#Definir diseño muestral para ambos años
dm<-enigh22%>%
  #Diseño muestral
  as_survey(weights=factor, strata=est_dis, ids=upm)

##  Habitabilidad

In [ ]:
print("Viviendas en rezago habitacional")
dm%>%
  group_by(rezago)%>%
  summarise(
    viviendas=survey_prop(vartype="cv"))%>%
    mutate(viviendas=round(viviendas*100,1))%>%
filter(rezago %in% c("En rezago"))

[1] "Viviendas en rezago habitacional"


rezago,viviendas,viviendas_cv
<chr>,<dbl>,<dbl>
En rezago,24.3,0.01054548


## Seguridad de la tenencia

In [ ]:
print("Viviendas propias y propias en proceso de pago sin escrituras")

dm%>%
#Filtrar propias y propias se están pagando
filter(tenencia == 3 | tenencia == 4)%>%
mutate(escrituras=case_when(
  escrituras==1~"A nombre del dueño",
  escrituras==2~"A nombre de otra persona",
    escrituras==3~"No tiene escrituras",
    escrituras==4~"No sabe si tiene escrituras",
))%>%
group_by(escrituras)%>%
  summarise(
    viviendas=survey_prop(vartype="cv"))%>%
    mutate(viviendas=round(viviendas*100,1))%>%
    filter(escrituras %in% c("No tiene escrituras"))

[1] "Viviendas propias y propias en proceso de pago sin escrituras"


escrituras,viviendas,viviendas_cv
<chr>,<dbl>,<dbl>
No tiene escrituras,24.3,0.01473525


## Ubicación

Para este indicador se utilizará la información del registro de vivienda que se publica en el Sistema Nacional de Información e Indicadores de Vivienda (SNIIV) de la Secretaría de Desarrollo Agrario, Territorial y Urbano (SEDATU).

In [ ]:
print("Viviendas fuera de contorno")

url<-"https://sniiv.sedatu.gob.mx/api/CuboAPI/GetRegistro/2022,2022/0/0/pcu,anio"

ubicacion<-GET(url)
ubicacion<-content(ubicacion,as="text")
ubicacion<-fromJSON(ubicacion)%>%
#Calcular porcentajes
mutate(

#Pcu numérico
viviendas=as.numeric(viviendas),
  porcentaje=round((viviendas/sum(viviendas))*100,1)
)%>%
filter(pcu %in% c("FC"))

ubicacion

[1] "Viviendas fuera de contorno"


año,pcu,viviendas,porcentaje
<int>,<chr>,<dbl>,<dbl>
2022,FC,21480,15


## Disponibilidad de servicios, materiales, instalaciones e infraestructura

In [ ]:
Sys.setlocale("LC_ALL", "en_US.UTF-8")

url_coneval<-"https://www.coneval.org.mx/Medicion/MP/Documents/MMP_2022/Programas_calculo/R_MMP_2022.zip"
download_and_unzip(url_coneval, "microdatos")

[1] "LC_COLLATE=en_US.UTF-8;LC_CTYPE=en_US.UTF-8;LC_MONETARY=en_US.UTF-8;LC_NUMERIC=C;LC_TIME=en_US.UTF-8"

ERROR: Error in unzip(destfile, exdir = dir): invalid multibyte string at '<a0>lcu'


In [ ]:
pobreza<-read.csv("microdatos/Base final/pobreza_22.csv")%>%
select(folioviv, foliohog,numren,ic_sbv)

str(pobreza)

'data.frame':	309534 obs. of  4 variables:
 $ folioviv: num  1e+08 1e+08 1e+08 1e+08 1e+08 ...
 $ foliohog: int  1 1 1 1 1 1 1 1 1 1 ...
 $ numren  : int  1 2 3 1 2 1 2 3 1 2 ...
 $ ic_sbv  : int  0 0 0 0 0 0 0 0 0 0 ...


In [ ]:
vivsb<-enigh22%>%
  left_join(
    pobreza %>%
      group_by(folioviv) %>%
      filter(row_number() == 1) %>%
      select(folioviv,ic_sbv),
    by = "folioviv"
  )

In [ ]:
dm_viv<-vivsb%>%
  as_survey(weights=factor, strata=est_dis, ids=upm)

In [ ]:
dm_viv%>%
group_by(ic_sbv)%>%
    summarise(hogares=survey_prop(vartype="ci"))%>%
    mutate(hogares=round(hogares*100,1))%>%
    filter(ic_sbv %in% c(1))

When `proportion` is unspecified, `survey_prop()` now defaults to `proportion = TRUE`.
ℹ This should improve confidence interval coverage.
This message is displayed once per session.


ic_sbv,hogares,hogares_low,hogares_upp
<int>,<dbl>,<dbl>,<dbl>
1,16.2,0.1566695,0.1669217
